In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('data/8-gon-gh-dataset.csv')

good_data = pd.isnull(df['dum1'])

dataset = df[good_data].dropna(axis=1, how='all')

# Split dataset into 70/15/15 training/validation/test
train = dataset.sample(frac=0.85)
test = dataset.drop(train.index)

# Split dataset into features and labels; last 4 (grid scores)
train_features = train.iloc[:, 1:]
train_labels = train.iloc[:, :1]

test_features = test.iloc[:, 1:]
test_labels = test.iloc[:, :1]

train_features.shape[0]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import os, datetime

model_path = "model/mesh-quality-8gon-small-1"
BATCH_SIZE = 10
EPOCHS = 1000
INITIAL_LEARNING_RATE = 1e-1

def model_setup():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(22,)),
        layers.BatchNormalization(),

        layers.Dense(16, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),

        layers.Dense(1),
    ])
    return model

def train_model(model, train_features, train_label, model_epochs=EPOCHS, model_batch_size=BATCH_SIZE, learning_rate=INITIAL_LEARNING_RATE):
    # Callbacks
    # logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    # tensorboard = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=EPOCHS//10, min_delta=0.0001)

    # Learning rate scheduler
    decay_steps= 10_000
    lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
        learning_rate, 
        decay_steps, 
        1e-2
        )

    # Initialize model
    model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(
                learning_rate=3e-4
            ),
            )

    # Train model and write progress to history (for graphing)
    history = model.fit(train_features,
                        train_labels,
                        epochs=model_epochs,
                        batch_size=model_batch_size,
                        validation_split=0.18,
                        verbose=2,
                        callbacks=[checkpoint, early_stopping],
                        )
    return history

model = model_setup()
model.summary()
history = train_model(model, train_features, train_labels)

# ======================
#       EVALUATION
# ======================
train_acc = model.evaluate(
    train_features, train_labels, verbose=0)
test_acc = model.evaluate(
    test_features, test_labels, verbose=0)
print('Training data loss: %.3f, Test data loss: %.3f' %
        (train_acc, test_acc))   

In [ ]:
max_test = [0.91484244,  0.00551026,  0.77336363,  0.79442204,
        0.06119724,  0.99685841, -0.48074287,  0.39218434, -0.9447753 ,
        0.17089053, -0.75703453, -0.93550632,  0.08711492, -0.97169181,
        0.73734201, -0.82820086, -0.25672259, -0.23196948,  0.22315476,
        0.27751235,  0.34114631, -0.34976791]
min_test = [1.4360157 , -0.28543287,  0.89610679,  0.17926611,
        0.00860781,  1.36589249, -0.40462255,  0.49788496, -0.97526413,
       -0.33061286, -0.70153368, -0.37833038,  0.43591692, -0.52997161,
        0.71686403, -0.25175814, -0.41137586, -0.18525747,  0.16669735,
        0.68101452,  0.66109099,  0.03499002]
test_label = 0.832




test_df = pd.read_csv('data/quality-test-big-dataset.csv')
good_data = pd.isnull(test_df['dum1'])

dataset = test_df[good_data].dropna(axis=1, how='all')


pred_model = tf.keras.models.load_model("model/mesh-quality-8gon-small-1")
features = np.expand_dims(max_test, axis=0)
features_min = np.expand_dims(min_test, axis=0)

min_pred = pred_model(features).numpy()
max_pred = pred_model(features_min).numpy()
# dataset['avgQuality'].idxmin()
# dataset.loc[5079].values
# dataset.head(100)
min_pred, max_pred

max_qual = 0.845 
min_qual = 0.401

0.6430032
0.6430032
0.6429662